# Semantic Kernel Tutorial: Building AI Agents for Insurance

This tutorial introduces **Semantic Kernel** by Microsoft, an SDK for integrating LLMs into applications using a "kernel" architecture with plugins and functions.

## What You'll Learn

1. Semantic Kernel concepts: Kernel, Plugins, and Functions
2. Building Weather and Eligibility Agents
3. Using Semantic Functions (prompt templates)
4. DSPy Integration
5. MLFlow Tracking

## 1. Installation & Setup

In [ ]:
# !pip install semantic-kernel httpx beautifulsoup4 python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE", "https://api.openai.com/v1")
model_name = os.getenv("MODEL_NAME", "gpt-4o-mini")

print(f"Model: {model_name}")

## 2. Core Concepts

- **Kernel**: Central orchestrator
- **Plugins**: Collections of functions
- **Native Functions**: Python functions
- **Semantic Functions**: LLM prompt templates

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

# Create kernel
kernel = sk.Kernel()

# Add chat service
kernel.add_service(
    OpenAIChatCompletion(
        service_id="chat",
        ai_model_id=model_name,
        api_key=api_key,
        base_url=api_base if api_base != "https://api.openai.com/v1" else None
    )
)

print("Kernel created")

## 3. Define Native Functions (Tools)

In [ ]:
import httpx
from bs4 import BeautifulSoup
from semantic_kernel.functions import kernel_function

class WeatherPlugin:
    """Plugin for weather verification."""
    
    @kernel_function(name="geocode", description="Convert location to coordinates")
    def geocode(self, location: str) -> str:
        try:
            with httpx.Client() as client:
                r = client.get(
                    "https://nominatim.openstreetmap.org/search",
                    params={"q": f"{location}, Australia", "format": "json", "limit": 1},
                    headers={"User-Agent": "InsuranceBot/1.0"},
                    timeout=10.0
                )
                if r.status_code == 200 and r.json():
                    d = r.json()[0]
                    return f"Location: {d['display_name']}\nLat: {d['lat']}\nLon: {d['lon']}"
                return f"Error: Location not found"
        except Exception as e:
            return f"Error: {e}"
    
    @kernel_function(name="get_weather", description="Get BOM weather data")
    def get_weather(self, latitude: str, longitude: str, date: str) -> str:
        try:
            year, month, day = date.split("-")
            url = "https://reg.bom.gov.au/cgi-bin/climate/storms/get_storms.py"
            params = {
                "begin_day": day, "begin_month": month, "begin_year": year,
                "end_day": day, "end_month": month, "end_year": year,
                "lat": float(latitude), "lng": float(longitude),
                "event": "all", "distance_from_point": "50", "states": "all"
            }
            
            with httpx.Client() as client:
                r = client.get(url, params=params, timeout=15.0)
                if r.status_code != 200:
                    return f"Error: HTTP {r.status_code}"
                
                soup = BeautifulSoup(r.text, 'html.parser')
                events = [c[0].get_text(strip=True) for row in soup.find_all('tr')[1:]
                         if (c := row.find_all('td')) and len(c) >= 2 and c[0].get_text(strip=True)]
                
                has_thunder = any('thunder' in e.lower() or 'lightning' in e.lower() for e in events)
                has_wind = any('wind' in e.lower() or 'gust' in e.lower() for e in events)
                
                return f"""Date: {date}
Events: {', '.join(events) if events else 'None'}
Has Thunderstorm: {has_thunder}
Has Strong Wind: {has_wind}"""
        except Exception as e:
            return f"Error: {e}"

# Add plugin
kernel.add_plugin(WeatherPlugin(), "weather")
print("Weather plugin added")

In [ ]:
# Test functions
weather_plugin = WeatherPlugin()

geo_result = weather_plugin.geocode("Brisbane, QLD")
print("Geocode result:")
print(geo_result)

weather_result = weather_plugin.get_weather("-27.4698", "153.0251", "2025-03-07")
print("\nWeather result:")
print(weather_result)

## 4. Define Semantic Functions

In [ ]:
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings

# Weather report semantic function
weather_prompt = """You are a Weather Verification Agent.

Given this data:
Location: {{$location_data}}
Weather: {{$weather_data}}

Create a structured weather verification report including:
- Location and coordinates
- Date
- Events found
- Thunderstorm status
- Strong wind status"""

weather_func = kernel.add_function(
    function_name="generate_report",
    plugin_name="reports",
    prompt=weather_prompt,
    prompt_execution_settings=OpenAIChatPromptExecutionSettings(
        service_id="chat",
        max_tokens=500
    )
)

print("Weather report function added")

In [ ]:
# Eligibility semantic function
eligibility_prompt = """You are a Claims Eligibility Agent.

Rules:
- APPROVED: Both thunderstorms AND strong winds detected in Australia
- REVIEW: Only one severe weather type detected
- DENIED: No severe weather or outside Australia

Weather Report:
{{$weather_report}}

Provide:
1. DECISION: APPROVED, REVIEW, or DENIED
2. REASONING: Brief explanation
3. CONFIDENCE: High, Medium, or Low"""

eligibility_func = kernel.add_function(
    function_name="determine_eligibility",
    plugin_name="claims",
    prompt=eligibility_prompt,
    prompt_execution_settings=OpenAIChatPromptExecutionSettings(
        service_id="chat",
        max_tokens=300
    )
)

print("Eligibility function added")

## 5. Run Pipeline

In [ ]:
async def process_claim(location: str, date: str):
    """Process claim through Semantic Kernel."""
    
    print("=" * 60)
    print(f"Processing: {location} on {date}")
    print("=" * 60)
    
    # Step 1: Geocode
    plugin = WeatherPlugin()
    geo_result = plugin.geocode(location)
    print(f"\nGeocoded:\n{geo_result}")
    
    # Extract lat/lon
    lat, lon = None, None
    for line in geo_result.split('\n'):
        if line.startswith('Lat:'):
            lat = line.split(':')[1].strip()
        elif line.startswith('Lon:'):
            lon = line.split(':')[1].strip()
    
    if not lat or not lon:
        return {"error": "Failed to geocode"}
    
    # Step 2: Get weather
    weather_data = plugin.get_weather(lat, lon, date)
    print(f"\nWeather:\n{weather_data}")
    
    # Step 3: Generate report
    report_result = await kernel.invoke(
        weather_func,
        location_data=geo_result,
        weather_data=weather_data
    )
    weather_report = str(report_result)
    print(f"\nReport:\n{weather_report}")
    
    # Step 4: Eligibility
    elig_result = await kernel.invoke(
        eligibility_func,
        weather_report=weather_report
    )
    
    print("\n" + "=" * 60)
    print("DECISION:")
    print("=" * 60)
    print(str(elig_result))
    
    return {
        "weather_report": weather_report,
        "decision": str(elig_result)
    }

# Run
result = await process_claim("Brisbane, QLD", "2025-03-07")

---

## 6. DSPy Integration

In [ ]:
import dspy

dspy_lm = dspy.LM(model=f"openai/{model_name}", api_key=api_key, api_base=api_base)
dspy.configure(lm=dspy_lm)

class EligSig(dspy.Signature):
    """CAT event eligibility."""
    weather_report: str = dspy.InputField()
    decision: str = dspy.OutputField(desc="APPROVED/REVIEW/DENIED")
    reasoning: str = dspy.OutputField()

elig_mod = dspy.ChainOfThought(EligSig)

# Quick test
r = elig_mod(weather_report="Brisbane. Thunder, Wind. Has Thunderstorm: True. Has Strong Wind: True.")
print(f"DSPy: {r.decision}")

In [ ]:
# Optimize
from dspy.teleprompt import BootstrapFewShot
from dspy.evaluate import Evaluate

examples = [
    dspy.Example(weather_report="Thunder+Wind. Both: True", decision="APPROVED", reasoning="Both met").with_inputs("weather_report"),
    dspy.Example(weather_report="Rain only. Both: False", decision="DENIED", reasoning="No severe").with_inputs("weather_report"),
    dspy.Example(weather_report="Thunder only. Wind: False", decision="REVIEW", reasoning="One met").with_inputs("weather_report"),
]

metric = lambda ex, pred, trace=None: ex.decision.upper() == pred.decision.upper()
optimizer = BootstrapFewShot(metric=metric, max_bootstrapped_demos=2)
optimized_mod = optimizer.compile(elig_mod, trainset=examples)

print("DSPy module optimized")

## 7. MLFlow Integration

In [ ]:
import mlflow
from datetime import datetime

mlflow.set_experiment("semantic-kernel-claims")

async def run_tracked(location: str, date: str):
    with mlflow.start_run(run_name=f"sk_{location.split(',')[0]}"):
        mlflow.log_params({"framework": "semantic-kernel", "model": model_name, "location": location})
        
        start = datetime.now()
        result = await process_claim(location, date)
        duration = (datetime.now() - start).total_seconds()
        
        decision = "UNKNOWN"
        for d in ["APPROVED", "DENIED", "REVIEW"]:
            if d in result.get("decision", "").upper():
                decision = d
                break
        
        mlflow.log_metrics({"duration": duration})
        mlflow.set_tags({"decision": decision})
        
        print(f"Logged: {decision}, {duration:.2f}s")
        return result

await run_tracked("Brisbane, QLD", "2025-03-07")

## 8. Summary

### Covered
- Kernel, plugins, native/semantic functions
- Tool definition via `@kernel_function`
- Prompt templates
- DSPy + MLFlow integration

### Strengths
- Enterprise-ready (Microsoft)
- Clear separation of concerns
- Good for .NET/C# integration

### Challenges
- More verbose than simpler frameworks

In [ ]:
print("Tutorial complete!")